# 6章 学習に関するテクニック<165>

## 6.1 パラメータの更新<165>

NNの学習の目的は、損失関数の値をできるだけ小さくするパラメータを発見すること。

これは、最適なパラメータを見つける問題で、そのような問題を解くことを**最適化**という。

NNではパラメータが膨大すぎるが、やみくもに求めるよりSGDを用いて計算することで効率的に最適なパラメータに向かっていたね。

だがしかし、SGDにも欠点はある。SGDの欠点を指摘しつつ、別の最適化手法を紹介しよう。

### 6.1.1 冒険家の話

### 6.1.2 SGD<166>

### 6.1.3 SGDの欠点<168>

SGDが非効率な例を挙げる

以下の式(6.2)の最小値を求める問題を考える

$$f(x,y) = \frac{1}{20}x^{2} + y^{2} \hspace{10mm}(6.2)$$

このグラフはお椀をx軸方向に伸ばしたような形。

こいつの勾配を求めてみると以下図のようになる

多くの場所で最小値(0,0)を刺さない悲しい感じ。

実際に(-7,2)を初期値としてSGDを適応してみると以下のようになる。


これからSGDの欠点は、関数が伸びた形だと、勾配が本来の最小値方向を示さず、非効率な経路で探索することになる。

### 6.1.4 Momentum<170>

SGDに代わる手法の一つ。Momentumとは、「運動量」という意味。

以下の数式で表される

$$\mathbf{v} \leftarrow \alpha\mathbf{v} - \eta \frac{\partial L}{\partial W} \hspace{10mm} (6.3)$$
$$W \leftarrow W + \mathbf{v} \hspace{10mm}(6.4)$$

$W$は更新する重みパラメータ

$\frac{\partial L}{\partial W}$は、$W$に関する損失関数の勾配

$\eta$は学習係数

$\mathbf{v}$は「物理」でいうところの速度

(6.3)式中にある$\alpha\mathbf{v}$は、物体が何の力も受けない時に徐々に減速していく役割。(空気抵抗や摩擦みたいなもん)

In [ ]:
class Momentum:
  def __init_(self, lr=0.01, mometum=0.9):
    self.lr = lr
    self.momentum = momentum
    self.v = None

  def update(self, params, grads):
    if self.v is None:
      self.v = {}
      for key, val in params.item():
        self.v[key] = np.zeros_like(val)
      
    for key in params.keys():
      self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]

class Momentum:

### 6.1.5 AdaGrad<172>

学習係数が小さいと、学習に時間がかかりすぎる

学習係数が大きいと、発散して正しく学習ができない。

この、学習係数に関するテクニックに**学習係数の減衰**というものがある。

最初は大きく、徐々に小さくして行くという方法である。

AdaGradでは、「全体」の学習係数を一括して小さくするのではなく、「ひとつひとつ」のパラメータに対してちいさくする。オーダーメイド



AdaGradの更新方法は以下の数式でかける。

$$h ← h + \frac{\partial L}{\partial W} \textcircled{・} \frac{\partial L}{\partial W} \hspace{10mm}(6.5)$$
$$W ← W - \eta \frac{1}{\sqrt{h}} \frac{\partial L}{\partial W} \hspace{10mm} (6.6)$$

$h$はこれまで経験した勾配の値を２乗和として保持。

パラメータ更新の際に$\frac{1}{\sqrt{h}}$をかけることでスケールを調整している。これにより、パラメータの要素の中でもよく動いた要素は学習係数がちいさくなる。

In [ ]:
class AdaGrad:
  def __init__(self, lr=0.01):
    self.lr = lr
    self.h = None

  def update(self, prams, grad):
    if self.h is None:
      self.f = {}
      for key, val in params.items():
        self.h[key] = np.zeros_like(val)

      for key in paramas.keys():
        self.h[key] += grads[key] * grads[key]
        params[key] -= self.lr * grads[key] /(np.sqrt(self.h[key]) + 1e-7)

圖６−６

### 6.1.6 Adam<175>

MomentumとAdaGradを融合するとどうなるのだろうか？それが、Adamのベースアイディア。



### 6.1.7 どの更新手法を用いるか？<175>

SGD,Momentum、AdamGrad,Adamとあったが、それぞれの得手不得手があるからどれがいいとかは一概にはいえない

### 6.1.8 Mnistデータセットによる更新手法の比較

ず６−８

## 6.2重みの初期化値<178>

### 6.2.1 重みの初期値を0にする？<178>

重みの初期値を０にするのは悪いアイディア。というか、初期値を同じ値にしておくことはとても良くないこと。なぜかというと、誤差逆伝播の時に全ての重みが同じように更新されてしまうので、重みが対称的な同じ値となり、多くの重みを用意する意味がなくなってしまうから。

### 6.2.2隠れ層のアクティベーション分布<179>

隠れ層の活性化関数かました後の分布を観察しよう。

それぞれの層は１００個のニューロンを持ち、入力データとして１０００個のデータをガウス分布でランダムに生成した５層NNに流します。

活性化関数にはシグモイド関数を使用。重みのスケールは標準偏差１のガウス分布を用いている。

するとず６−１０の結果が得られる

標準偏差を0.01にすると以下の分布になる。

では、どんな重みがいいか？「Xavierの初期値」ていうやつが一般的なDLのフレームワークで用いられている。この結果がず６−１３